# ラズベリーパイのセンサ制御の基礎

ラズベリーパイには下の図のように40個のピンがついており，これらをセンサーに繋ぎます。
(ピン番号対応表はラズベリーパイ付属のカードにも記載されています。)
ピン位置に書いてある1-40の番号は物理ピンと呼ばれております。
それぞれのピンの用途が横に書かれてあり，主に以下に示す3つの役割があります。
- power
- Ground
- GPIO

それぞれの役割を以下で簡単に説明します。

<img src="gpio_pics/GPIO-Pinout-Diagram-2.png" width="800">

## Powerについて
5V powerからは+5 Vで一定の電圧が出続けています。センサの電源として5 Vが必要な場合はこのピンを使ってください。

3V3 powerからは+3.3 Vで一定の電圧が出続けています。センサの電源として3.3 Vが必要な場合はこのピンを使ってください。

## Groundについて
電圧(電気の圧力)は絶対的な値ではありません。

上で説明した+5 Vや+3.3 Vは，それぞれどこに対しての電圧であるかを示すための0 Vの基準が必要です。

0 Vの基準をGroundと呼びます。

センサに5 Vの入力が必要な場合は，Groundと5 Vの線をそれぞれ繋ぐ必要があります。

## GPIIOについて
センサを制御しているピンがGPIOと呼ばれるものです。

センサにシグナルを送ること，またセンサからのシグナルを受け取ることができます。

GPIOピンを使うときはまず，シグナルを送るため(OUTPUT)に使うのか，それとも受け取るため(INPUT)に使うのかを指定する必要があります。

- OUTPUTとして使う場合：

3.3 Vの出力のオン／オフを素早く切り替えることができます。

- INPUTとして使う場合：

1.3 ~ 3.3 Vの電圧が入力されると"シグナルあり"，それ以下の電圧では"シグナルなし"とみなされます。

例えば2 Vの電圧が入力された場合は"シグナルあり"とみなされるだけであり，2 Vとして読むことはできません。

---

## Node-Redの基礎
injectノードとdebugノードを使いましょう。

injectノードはプログラムをスタートさせるノード，debugノードはプリントだと思ってください。

<img src="gpio_pics/nr1.png" width="800">

[{"id":"b436f784.ce6928","type":"inject","z":"c9390f4.5f064f","name":"","props":[{"p":"payload"}],"repeat":"","crontab":"","once":false,"onceDelay":0.1,"topic":"","payload":"0","payloadType":"num","x":230,"y":40,"wires":[["b9931ace.b1d218"]]},{"id":"b9931ace.b1d218","type":"debug","z":"c9390f4.5f064f","name":"","active":true,"tosidebar":true,"console":false,"tostatus":false,"complete":"payload","targetType":"msg","statusVal":"","statusType":"auto","x":510,"y":40,"wires":[]}]

## GPIOから出力
GPIO03(5番ピン)にLEDをつなぎ，GPIO03に0, 1を送ってLEDを点滅させましょう。

<img src="gpio_pics/led1.png" width="600">

[{"id":"3531e954.6326f6","type":"inject","z":"c9390f4.5f064f","name":"","props":[{"p":"payload"},{"p":"topic","vt":"str"}],"repeat":"","crontab":"","once":false,"onceDelay":0.1,"topic":"","payload":"1","payloadType":"num","x":270,"y":200,"wires":[["34076d2.f401f92"]]},{"id":"debcc4f7.e80c48","type":"inject","z":"c9390f4.5f064f","name":"","props":[{"p":"payload"},{"p":"topic","vt":"str"}],"repeat":"","crontab":"","once":false,"onceDelay":0.1,"topic":"","payload":"0","payloadType":"num","x":270,"y":240,"wires":[["34076d2.f401f92"]]},{"id":"5b8a9d60.b3ffc4","type":"comment","z":"c9390f4.5f064f","name":"LEDへのON/OFF出力","info":"","x":250,"y":160,"wires":[]},{"id":"34076d2.f401f92","type":"rpi-gpio out","z":"c9390f4.5f064f","name":"","pin":"5","set":true,"level":"0","freq":"","out":"out","x":460,"y":220,"wires":[]}]

---

## GPIOに入力

GPIO05の出力をGPIO21につなぎ，GPIOへの入力を読み取りましょう。

<img src="gpio_pics/led2.png" width="600">

[{"id":"475c2f85.8d0d","type":"debug","z":"c9390f4.5f064f","name":"","active":false,"tosidebar":true,"console":false,"tostatus":false,"complete":"false","statusVal":"","statusType":"auto","x":440,"y":380,"wires":[]},{"id":"b4366f52.0d712","type":"comment","z":"c9390f4.5f064f","name":"LEDからのON/OFF入力","info":"","x":260,"y":320,"wires":[]},{"id":"c7002a1a.f6fe28","type":"rpi-gpio in","z":"c9390f4.5f064f","name":"","pin":"40","intype":"tri","debounce":"25","read":false,"x":230,"y":380,"wires":[["475c2f85.8d0d"]]}]

---

## 課題：
LEDを1秒ONにしてOFFにするフローを作りましょう。

<img src="gpio_pics/led3.png" width="800">

[{"id":"a8da80b7.b8997","type":"comment","z":"c9390f4.5f064f","name":"LEDを1秒ONにしてOFF","info":"","x":310,"y":500,"wires":[]},{"id":"8cc7f520.c96138","type":"inject","z":"c9390f4.5f064f","name":"スタート","props":[{"p":"payload"},{"p":"topic","vt":"str"}],"repeat":"","crontab":"","once":false,"onceDelay":0.1,"topic":"","payload":"","payloadType":"date","x":290,"y":580,"wires":[["16502ce9.24e773"]]},{"id":"4d875ad3.e6f374","type":"delay","z":"c9390f4.5f064f","name":"","pauseType":"delay","timeout":"1","timeoutUnits":"seconds","rate":"1","nbRateUnits":"1","rateUnits":"second","randomFirst":"1","randomLast":"5","randomUnits":"seconds","drop":false,"x":640,"y":580,"wires":[["8c676df.1bd859"]]},{"id":"16502ce9.24e773","type":"change","z":"c9390f4.5f064f","name":"payload = 1","rules":[{"t":"set","p":"payload","pt":"msg","to":"1","tot":"num"}],"action":"","property":"","from":"","to":"","reg":false,"x":470,"y":580,"wires":[["4d875ad3.e6f374","50161329.5c45cc"]]},{"id":"8c676df.1bd859","type":"change","z":"c9390f4.5f064f","name":"payload = 0","rules":[{"t":"set","p":"payload","pt":"msg","to":"0","tot":"num"}],"action":"","property":"","from":"","to":"","reg":false,"x":830,"y":580,"wires":[["50161329.5c45cc"]]},{"id":"50161329.5c45cc","type":"rpi-gpio out","z":"c9390f4.5f064f","name":"","pin":"5","set":true,"level":"0","freq":"","out":"out","x":720,"y":500,"wires":[]}]

---

## 課題：
LEDのONとOFFを1秒おきに2回繰り返すフローを作りましょう。

<img src="gpio_pics/led4.png" width="1200">

[{"id":"395dddf1.00c782","type":"comment","z":"c9390f4.5f064f","name":"LEDのONとOFFを1秒おきに2回繰り返す","info":"","x":360,"y":720,"wires":[]},{"id":"e7c47926.55c448","type":"inject","z":"c9390f4.5f064f","name":"スタート","props":[{"p":"payload"},{"p":"topic","vt":"str"}],"repeat":"","crontab":"","once":false,"onceDelay":0.1,"topic":"","payload":"","payloadType":"date","x":290,"y":820,"wires":[["707590de.5715b"]]},{"id":"942dfa95.d43468","type":"delay","z":"c9390f4.5f064f","name":"","pauseType":"delay","timeout":"1","timeoutUnits":"seconds","rate":"1","nbRateUnits":"1","rateUnits":"second","randomFirst":"1","randomLast":"5","randomUnits":"seconds","drop":false,"x":640,"y":820,"wires":[["f0fa3310.a2d87"]]},{"id":"13aae8c1.72fa57","type":"delay","z":"c9390f4.5f064f","name":"","pauseType":"delay","timeout":"1","timeoutUnits":"seconds","rate":"1","nbRateUnits":"1","rateUnits":"second","randomFirst":"1","randomLast":"5","randomUnits":"seconds","drop":false,"x":980,"y":820,"wires":[["c38b6486.4582f8"]]},{"id":"5d70cbb6.91c374","type":"delay","z":"c9390f4.5f064f","name":"","pauseType":"delay","timeout":"1","timeoutUnits":"seconds","rate":"1","nbRateUnits":"1","rateUnits":"second","randomFirst":"1","randomLast":"5","randomUnits":"seconds","drop":false,"x":1320,"y":820,"wires":[["9c964b60.317438"]]},{"id":"707590de.5715b","type":"change","z":"c9390f4.5f064f","name":"payload = 1","rules":[{"t":"set","p":"payload","pt":"msg","to":"1","tot":"num"}],"action":"","property":"","from":"","to":"","reg":false,"x":470,"y":820,"wires":[["942dfa95.d43468","38fec960.987916"]]},{"id":"f0fa3310.a2d87","type":"change","z":"c9390f4.5f064f","name":"payload = 0","rules":[{"t":"set","p":"payload","pt":"msg","to":"0","tot":"num"}],"action":"","property":"","from":"","to":"","reg":false,"x":810,"y":820,"wires":[["13aae8c1.72fa57","38fec960.987916"]]},{"id":"c38b6486.4582f8","type":"change","z":"c9390f4.5f064f","name":"payload = 1","rules":[{"t":"set","p":"payload","pt":"msg","to":"1","tot":"num"}],"action":"","property":"","from":"","to":"","reg":false,"x":1150,"y":820,"wires":[["5d70cbb6.91c374","38fec960.987916"]]},{"id":"9c964b60.317438","type":"change","z":"c9390f4.5f064f","name":"payload = 0","rules":[{"t":"set","p":"payload","pt":"msg","to":"0","tot":"num"}],"action":"","property":"","from":"","to":"","reg":false,"x":1490,"y":820,"wires":[["38fec960.987916"]]},{"id":"38fec960.987916","type":"rpi-gpio out","z":"c9390f4.5f064f","name":"","pin":"5","set":true,"level":"0","freq":"","out":"out","x":740,"y":700,"wires":[]}]

---

## 課題：
LEDのONとOFFを1秒おきにずっと繰り返すフローを作りましょう。

<img src="gpio_pics/led5.png" width="1200">

[{"id":"5648d631.cf5b98","type":"comment","z":"c9390f4.5f064f","name":"LEDのONとOFFを1秒おきにずっと繰り返す","info":"","x":370,"y":980,"wires":[]},{"id":"c8bc8987.406688","type":"inject","z":"c9390f4.5f064f","name":"スタート","props":[{"p":"payload"},{"p":"topic","vt":"str"}],"repeat":"","crontab":"","once":false,"onceDelay":0.1,"topic":"","payload":"","payloadType":"date","x":280,"y":1100,"wires":[["e644d772.cd6928"]]},{"id":"62ddfdf8.f0fc04","type":"delay","z":"c9390f4.5f064f","name":"","pauseType":"delay","timeout":"1","timeoutUnits":"seconds","rate":"1","nbRateUnits":"1","rateUnits":"second","randomFirst":"1","randomLast":"5","randomUnits":"seconds","drop":false,"x":680,"y":1100,"wires":[["254276bf.fc6d0a"]]},{"id":"77fdf7bb.c1afd8","type":"delay","z":"c9390f4.5f064f","name":"","pauseType":"delay","timeout":"1","timeoutUnits":"seconds","rate":"1","nbRateUnits":"1","rateUnits":"second","randomFirst":"1","randomLast":"5","randomUnits":"seconds","drop":false,"x":1080,"y":1100,"wires":[["e644d772.cd6928"]]},{"id":"e644d772.cd6928","type":"change","z":"c9390f4.5f064f","name":"payload = 1","rules":[{"t":"set","p":"payload","pt":"msg","to":"1","tot":"num"}],"action":"","property":"","from":"","to":"","reg":false,"x":490,"y":1100,"wires":[["62ddfdf8.f0fc04","5d5ee26c.0eb9dc"]]},{"id":"254276bf.fc6d0a","type":"change","z":"c9390f4.5f064f","name":"payload = 0","rules":[{"t":"set","p":"payload","pt":"msg","to":"0","tot":"num"}],"action":"","property":"","from":"","to":"","reg":false,"x":870,"y":1100,"wires":[["77fdf7bb.c1afd8","5d5ee26c.0eb9dc"]]},{"id":"5d5ee26c.0eb9dc","type":"rpi-gpio out","z":"c9390f4.5f064f","name":"","pin":"5","set":true,"level":"0","freq":"","out":"out","x":840,"y":1000,"wires":[]}]

---

## 課題：
LEDのONとOFFを1秒おきに5回繰り返すフローを作りましょう。

<img src="gpio_pics/led6.png" width="1200">

[{"id":"2e7af677.87727a","type":"comment","z":"c9390f4.5f064f","name":"LEDのONとOFFを1秒おきに5回繰り返す","info":"","x":380,"y":1360,"wires":[]},{"id":"269392ff.02ec5e","type":"inject","z":"c9390f4.5f064f","name":"スタート","props":[{"p":"payload"},{"p":"topic","vt":"str"}],"repeat":"","crontab":"","once":false,"onceDelay":0.1,"topic":"","payload":"","payloadType":"date","x":300,"y":1440,"wires":[["3b11ee0f.768f22"]]},{"id":"4d08a68c.768448","type":"delay","z":"c9390f4.5f064f","name":"","pauseType":"delay","timeout":"1","timeoutUnits":"seconds","rate":"1","nbRateUnits":"1","rateUnits":"second","randomFirst":"1","randomLast":"5","randomUnits":"seconds","drop":false,"x":1060,"y":1440,"wires":[["b786bd1b.6a0b6"]]},{"id":"71418636.43cda8","type":"delay","z":"c9390f4.5f064f","name":"","pauseType":"delay","timeout":"1","timeoutUnits":"seconds","rate":"1","nbRateUnits":"1","rateUnits":"second","randomFirst":"1","randomLast":"5","randomUnits":"seconds","drop":false,"x":1460,"y":1440,"wires":[["718426b3.477498"]]},{"id":"3b11ee0f.768f22","type":"change","z":"c9390f4.5f064f","name":"LC=0","rules":[{"t":"set","p":"LC","pt":"msg","to":"0","tot":"num"}],"action":"","property":"","from":"","to":"","reg":false,"x":470,"y":1440,"wires":[["13c9a0b4.2011af"]]},{"id":"13c9a0b4.2011af","type":"switch","z":"c9390f4.5f064f","name":"ループのif文","property":"LC","propertyType":"msg","rules":[{"t":"lt","v":"5","vt":"num"}],"checkall":"true","repair":false,"outputs":1,"x":630,"y":1440,"wires":[["1f541fb9.866bd"]]},{"id":"718426b3.477498","type":"function","z":"c9390f4.5f064f","name":"LC = LC + 1","func":"msg.LC = msg.LC + 1;\nreturn msg;","outputs":1,"noerr":0,"initialize":"","finalize":"","x":1660,"y":1440,"wires":[["13c9a0b4.2011af"]]},{"id":"1f541fb9.866bd","type":"change","z":"c9390f4.5f064f","name":"payload = 1","rules":[{"t":"set","p":"payload","pt":"msg","to":"1","tot":"num"}],"action":"","property":"","from":"","to":"","reg":false,"x":850,"y":1440,"wires":[["4d08a68c.768448","c72ced4.993b71"]]},{"id":"b786bd1b.6a0b6","type":"change","z":"c9390f4.5f064f","name":"payload = 0","rules":[{"t":"set","p":"payload","pt":"msg","to":"0","tot":"num"}],"action":"","property":"","from":"","to":"","reg":false,"x":1250,"y":1440,"wires":[["71418636.43cda8","c72ced4.993b71"]]},{"id":"c72ced4.993b71","type":"rpi-gpio out","z":"c9390f4.5f064f","name":"","pin":"5","set":true,"level":"0","freq":"","out":"out","x":1240,"y":1340,"wires":[]}]